# Basics on Markov chain Monte Carlo and Physics applications
## Metropolis-(Hastings) algorithm
The Metropolis-Hastings algorithm is the most important and most popular tool in Markov chain Monte Carlo (MCMC), Bayesian statistics, and physics simulations. It is universal and applicable when you wish to generate any kinds of distributions. The other methods usually have some restrictions, so if you do not know how to use another one like a Gibbs sampler, you should choose the Metropolis-Hastings method.

### Classical Monte Carlo simulation for Ising model
As a typical example, I will present a simple Julia implimentation of the Monte Carlo simuplation of the 2D Ising model on the square lattice. The Metropolis rule is simply give by the following function.